In [ ]:
from itertools import product

import plotly.express as px
import polars as pl

In [ ]:
traps = list(range(1, 7))
potential_outcomes = list(product(traps, traps, traps, traps, traps, traps))

In [ ]:
def near(a, b):
    return abs(a - b) < 1e-6

In [ ]:
def generate_hi_lo_sequence(outcome):
    last_trap = 3.5

    result = []

    for trap in outcome:
        if near(trap, last_trap):
            result.append("E")
        elif trap < last_trap:
            result.append("L")
        else:
            result.append("H")

        last_trap = trap

    return result

In [ ]:
hi_lo_sequence_df = pl.DataFrame({
    "outcome": ["".join(str(trap) for trap in outcome) for outcome in potential_outcomes],
    "hi_lo_sequence": ["".join(generate_hi_lo_sequence(outcome)) for outcome in potential_outcomes]
})

In [ ]:
hi_lo_sequence_df.write_parquet("hi-lo-sequencies.parquet")

In [ ]:
hi_lo_sequence_frequency_df = (
    hi_lo_sequence_df
    .select([pl.col("hi_lo_sequence").value_counts(sort=True)])
    .unnest("hi_lo_sequence")
)

In [ ]:
fig = px.bar(
    hi_lo_sequence_frequency_df,
    x="count",
    y="hi_lo_sequence",
    orientation="h",
    height=1000,
    labels={
        "hi_lo_sequence": "Sequence",
        "count": "n",
    },
)

fig.show()